In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
load_dotenv()


True

In [26]:
from langchain_core.tools import InjectedToolArg, tool
from typing import Annotated

@tool #tool1
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool  # tool 2
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

#test without annotated
# @tool  # tool 2
# def convert(base_currency_value: int, conversion_rate: float) -> float:
#   """
#   given a currency conversion rate this function calculates the target currency value from a given base currency value
#   """

#   return base_currency_value * conversion_rate

In [27]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [28]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756252801,
 'time_last_update_utc': 'Wed, 27 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1756339201,
 'time_next_update_utc': 'Thu, 28 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.6692}

In [29]:
convert.invoke({'base_currency_value':10, 'conversion_rate':87.6692})    

876.692

In [30]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [31]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [32]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [33]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [34]:
ai_message = llm_with_tools.invoke(messages)

In [35]:
messages.append(ai_message)

In [36]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': '1c7f512e-20ff-41ad-98c2-9d222de86c87',
  'type': 'tool_call'}]

In [37]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [38]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--91d5de3f-cd0b-449c-b3e7-a0ba623e6098-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': '1c7f512e-20ff-41ad-98c2-9d222de86c87', 'type': 'tool_call'}], usage_metadata={'input_tokens': 150, 'output_tokens': 26, 'total_tokens': 631, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756252

In [39]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01141.\nBased on that, 10 INR is equal to 0.1141 USD.'